<a href="https://colab.research.google.com/github/CS23M005/Assignment2_PartA/blob/main/CS23M005_A2_PartA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np
import math

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
device

device(type='cpu')

In [22]:
!wget https://storage.googleapis.com/wandb_datasets/nature_12K.zip -O nature_12K.zip
!unzip -q nature_12K.zip

--2024-04-03 17:19:54--  https://storage.googleapis.com/wandb_datasets/nature_12K.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.201.207, 74.125.69.207, 64.233.181.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.201.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3816687935 (3.6G) [application/zip]
Saving to: ‘nature_12K.zip’

nature_12K.zip      100%[===================>]   3.55G  73.2MB/s    in 49s     

2024-04-03 17:20:43 (74.5 MB/s) - ‘nature_12K.zip’ saved [3816687935/3816687935]



In [23]:
!rm nature_12K.zip

In [24]:
from torchvision.datasets import ImageFolder
import torchvision.transforms as transforms
from torch.utils.data import Subset, DataLoader
import torchvision
from torch import optim
from tqdm import tqdm
from torch.utils.data import DataLoader
from torchvision import datasets

transform = transforms.Compose([
    transforms.Resize((256,256)),
    transforms.ToTensor(),
    transforms.Normalize((0.5,),(0.5,))
])

train_dataset = datasets.ImageFolder(root='inaturalist_12K/train',transform=transform)
train_dataset,val_dataset = torch.utils.data.random_split(train_dataset,[8000,1999])

train_loader = torch.utils.data.DataLoader(train_dataset,
    batch_size =32,shuffle = True,num_workers=2,pin_memory=True)

test_data = torch.utils.data.DataLoader(
    datasets.ImageFolder(root='inaturalist_12K/val',transform=transform),batch_size = 32,shuffle = True,num_workers=2,pin_memory=True
)

In [20]:
class ConvNet(nn.Module):
    def __init__(self, input_channel, output_size, num_filters, filter_size, activation_fun, filter_config, maxPoolStride, stride, neurons):
        super(ConvNet,self).__init__()
        self.k = filter_size
        m = self.getM()
        self.s = stride
        w, h = 256,256

        self.conv1 = nn.Conv2d(input_channel, m[0], self.k, self.s)
        w,h = self.getWH(w,h)
        self.pool1 = nn.MaxPool2d(self.k,self.s)
        w,h = self.getWH(w,h)
        self.conv2 = nn.Conv2d(m[0], m[1], self.k, self.s)
        w,h = self.getWH(w,h)
        self.pool2 = nn.MaxPool2d(self.k, self.s)
        w,h = self.getWH(w,h)
        self.conv3 = nn.Conv2d(m[1], m[2], self.k, self.s)
        w,h = self.getWH(w,h)
        self.pool3 = nn.MaxPool2d(self.k, self.s)
        w,h = self.getWH(w,h)
        self.conv4 = nn.Conv2d(m[2], m[3], self.k, self.s)
        w,h = self.getWH(w,h)
        self.pool4 = nn.MaxPool2d(self.k, self.s)
        w,h = self.getWH(w,h)
        self.conv5 = nn.Conv2d(m[3], m[4], self.k, self.s)
        w,h = self.getWH(w,h)
        self.pool5 = nn.MaxPool2d(self.k, self.s)
        w,h = self.getWH(w,h)
        self.fc1 = nn.Linear(m[4]*w*h, neurons)
        self.fc2 = nn.Linear(neurons,10)



    def getWH(self, w, h):
      return (math.floor(((w-self.k)/self.s)+1),math.floor(((w-self.k)/self.s)+1))

    def getM(self):
        m = []
        if(filter_config == "double"):
          for i in range(1,6):
            m.append((int)(2**(i)*num_filters))
        elif(filter_config == "same"):
          for i in range(1,6):
            m.append(int(num_filters))
        elif(filter_config == "half"):
          for i in range(1,6):
            m.append((int)(num_filters/(2**(i))))
        return m

    def getActivation_fn(self):
        if(activation_fun == "relu"):
            activation_fn = F.relu
        elif(activation_fun == "gelu"):
            activation_fn = F.gelu
        elif(activation_fun == "silu"):
            activation_fn = F.silu
        elif(activation_fun=="mish"):
            activation_fn = F.mish
        return activation_fn

    def forward(self, x):
        activation_fn = self.getActivation_fn()
        x = activation_fn(self.conv1(x))
        x = self.pool1(x)
        x = activation_fn(self.conv2(x))
        x = self.pool2(x)
        x = activation_fn(self.conv3(x))
        x = self.pool3(x)
        x = activation_fn(self.conv4(x))
        x = self.pool4(x)
        x = activation_fn(self.conv5(x))
        x = self.pool5(x)
        x = x.reshape(x.shape[0], -1)
        x = activation_fn(self.fc1(x))
        x = self.fc2(x)
        return x

x = torch.randn((1,3,256,256))
input_channel=3
output_size=10
num_filters=32
filter_size=3
activation_fun = "relu"
filter_config = "same"
maxPoolStride = 2
stride = 1
neurons= 128
model1 = ConvNet(input_channel, output_size, num_filters, filter_size, activation_fun, filter_config, maxPoolStride, stride, neurons).to(device)
y = model1(x)
print(y)

tensor([[ 0.1188, -0.0448,  0.0622, -0.0530, -0.0801, -0.0069, -0.0533, -0.0200,
          0.0986, -0.0430]], grad_fn=<AddmmBackward0>)


In [ ]:
!pip install wandb
import wandb
wandb.login()

In [ ]:
class ConvNet(nn.Module):
  def __init__(self, input_channel, output_size, num_filters, filter_size, activation_fun, filter_config):
    super(ConvNet,self).__init__()
    self.k = filter_size

    if(filter_config == "double"):
      self.m = 2*num_filters
    elif(filter_config == "half"):
      self.m = num_filters/2
    elif(filter_config == "same"):
      self.m = num_filters

    if(activation_fun=="ReLU"):
      self.layer2 = nn.ReLU()
    elif(activation_fun=="GELU"):
      self.layer2 = nn.GELU()
    elif(activation_fun == "SiLU"):
      self.layer2 = nn.SiLU()
    elif(activation_fun == "Mish"):
      self.layer2 = nn.Mish()

    self.dropout1 = nn.Dropout2d(0.25)
    self.dropout2 = nn.Dropout2d(0.5)
    self.fc1 = nn.Linear(9216, 128)
    self.fc2 = nn.Linear(128, 10)

    self.input_layer1 = nn.Conv2d(input_channel, self.m, self.k, 1)
    self.layer3 = nn.MaxPool2d(2,2)
    self.layer1 = nn.Conv2d(self.m, self.m, self.k)

  def forward(self,x):
    x = self.input_layer1(x)
    x=  self.layer2(x)
    x = self.layer3(x)
    x = nn.Flatten(x)
    return x
    #print(model)

x = torch.randn((10,3,28,28))
input_channel=3
output_size=10
num_filters=32
filter_size=3
activation_fun = "ReLU"
filter_config = "same"
model1 = ConvNet(input_channel, output_size, num_filters, filter_size, activation_fun, filter_config)
y = model1(x)
print(y)

Flatten(
  start_dim=tensor([[[[0.0141, 0.5200, 0.4232,  ..., 0.2180, 0.6843, 0.6098],
            [0.3145, 1.0824, 1.1824,  ..., 0.3572, 0.6627, 0.4583],
            [0.2714, 0.3871, 0.8526,  ..., 0.8004, 0.4321, 0.7791],
            ...,
            [0.6750, 0.0000, 0.4083,  ..., 0.8292, 0.5178, 0.7919],
            [1.0533, 0.1760, 0.3061,  ..., 1.6953, 0.8528, 0.6622],
            [0.6455, 0.3990, 0.4902,  ..., 0.3586, 1.2961, 0.3871]],
  
           [[0.7864, 0.9491, 0.9243,  ..., 0.7549, 0.3363, 1.2432],
            [0.0795, 0.2885, 0.6865,  ..., 0.9556, 0.5447, 0.5059],
            [0.9100, 1.4908, 1.2771,  ..., 0.7624, 1.0212, 0.8673],
            ...,
            [1.2515, 0.6684, 1.0149,  ..., 0.1129, 0.9377, 0.8494],
            [0.0547, 0.4853, 0.8530,  ..., 0.6049, 1.2547, 0.4948],
            [0.2218, 0.4209, 1.0938,  ..., 0.0026, 0.4970, 0.8769]],
  
           [[0.5306, 0.8761, 0.0849,  ..., 0.5633, 0.0000, 1.7139],
            [0.7806, 0.4060, 1.1231,  ..., 0.6462, 0.51

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

device = 'cuda:0'
inputs = [torch.zeros((1, 1, 2048, 2048)).to(device)]*2
weight = torch.ones((48, 1, 501, 501), dtype=torch.float32).to(device)
for (i, input) in enumerate(inputs):
    out = F.conv2d(input, weight)
    print(out.shape)
    print(torch.argmax(out))

torch.Size([1, 48, 1548, 1548])
tensor(0, device='cuda:0')
torch.Size([1, 48, 1548, 1548])
tensor(0, device='cuda:0')


In [ ]:
class Net(nn.Module):
    def __init__(self):
      super(Net, self).__init__()
      self.conv1 = nn.Conv2d(1, 32, 3, 1)
      self.conv2 = nn.Conv2d(32, 64, 3, 1)
      self.dropout1 = nn.Dropout2d(0.25)
      self.dropout2 = nn.Dropout2d(0.5)
      self.fc1 = nn.Linear(9216, 128)
      self.fc2 = nn.Linear(128, 10)

    # x represents our data
    def forward(self, x):
      # Pass data through conv1
      x = self.conv1(x)
      # Use the rectified-linear activation function over x
      x = F.relu(x)

      x = self.conv2(x)
      x = F.relu(x)

      # Run max pooling over x
      x = F.max_pool2d(x, 2)
      # Pass data through dropout1
      x = self.dropout1(x)
      # Flatten x with start_dim=1
      x = torch.flatten(x, 1)
      # Pass data through ``fc1``
      x = self.fc1(x)
      x = F.relu(x)
      x = self.dropout2(x)
      x = self.fc2(x)

      # Apply softmax to x
      output = F.log_softmax(x, dim=1)
      return output

In [ ]:
# Equates to one random 28x28 image
random_data = torch.rand((1, 1, 28, 28))

my_nn = Net()
result = my_nn(random_data)
print (result)

tensor([[-2.3844, -2.3163, -2.1654, -2.2708, -2.3411, -2.3325, -2.2021, -2.3257,
         -2.3507, -2.3600]], grad_fn=<LogSoftmaxBackward0>)


/usr/local/lib/python3.10/dist-packages/torch/nn/functional.py:1347: UserWarning: dropout2d: Received a 2-D input to dropout2d, which is deprecated and will result in an error in a future release. To retain the behavior and silence this warning, please use dropout instead. Note that dropout2d exists to provide channel-wise dropout on inputs with 2 spatial dimensions, a channel dimension, and an optional batch dimension (i.e. 3D or 4D inputs).
  warnings.warn(warn_msg)


In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

n_total_steps = len(train_loader)
for epoch in range(num_epochs):

    running_loss = 0.0

    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward and optimize
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        running_loss += loss.item()

    print(f'[{epoch + 1}] loss: {running_loss / n_total_steps:.3f}')

print('Finished Training')
PATH = './cnn.pth'
torch.save(model.state_dict(), PATH)

In [ ]:
def train_nn():
    config_defaults = {
            "num_epochs": 3, \
            "num_hidden_layers":  4, \
            "size_hidden_layer": 128, \
            "learning_rate": 1e-3, \
            "optimizer": "rmsprop", \
            "batch_size": 32, \
            "weight_init": "xavier" , \
            "activation": "tanh", \
            "loss": "crossentropy", \
            "reg_lambda": 0.001, \
    }
    wandb.init(config = config_defaults, project = "Assignment1")
    config = wandb.config
    print(config.optimizer)
    wandb.run.name = "e_{}_hl_{}_opt_{}_bs_{}_init_{}_ac_{}_loss_{}".format(config.num_epochs,\
                                                                  config.num_hidden_layers,\
                                                                  config.size_hidden_layer,\
                                                                  config.learning_rate,\
                                                                  config.optimizer,\
                                                                  config.batch_size,\
                                                                  config.weight_init,\
                                                                  config.activation,\
                                                                  config.loss,\
                                                                  config.reg_lambda)

    seed(42)
    n_inputs = 784
    n_layers = config.num_hidden_layers
    n_neurons = config.size_hidden_layer
    n_outputs = 10
    init_mode = config.weight_init
    w,b = init_nn(n_inputs, n_layers, n_neurons, n_outputs, init_mode)
    w_old = w.copy()
    b_old = b.copy()

    eta = config.learning_rate
    activation_f = config.activation
    batch_size = config.batch_size
    loss = config.loss
    ud_lambda = config.reg_lambda
    optimizer = config.optimizer

    beta = 0.9
    beta1 = 0.9
    beta2 = 0.999

    eps = 1e-9
    max_epochs = config.num_epochs
    momentum = 0.9



    wandb.log({"examples": fig1})



    if optimizer == "sgd":
      w_n,b_n, a_n, h_n = sgd(n_inputs, n_layers, n_neurons, n_outputs, activation_f, batch_size, loss, ud_lambda, eta, init_mode, w, b, True, max_epochs)
    elif optimizer == "mgd":
      w_n,b_n, a_n, h_n = mgd(n_inputs, n_layers, n_neurons, n_outputs, activation_f, batch_size, loss, ud_lambda, eta, init_mode, w, b, max_epochs, momentum)
    elif optimizer == "nag":
      w_n,b_n, a_n, h_n = nag(n_inputs, n_layers, n_neurons, n_outputs, activation_f, batch_size, loss, ud_lambda, eta, init_mode, w, b, True, max_epochs, momentum)
    elif optimizer == "rmsprop":
      w_n,b_n, a_n, h_n = rmsprop(n_inputs, n_layers, n_neurons, n_outputs, activation_f, batch_size, loss, ud_lambda, eta, init_mode, w, b, True, max_epochs, beta, eps)
    elif optimizer == "adam":
      w_n,b_n, a_n, h_n = adam(n_inputs, n_layers, n_neurons, n_outputs, activation_f, batch_size, loss, ud_lambda, eta, init_mode, w, b, True, max_epochs, beta1, beta2, eps)
    elif optimizer == "nadam":
      w_n,b_n, a_n, h_n = nadam(n_inputs, n_layers, n_neurons, n_outputs, activation_f, batch_size, loss, ud_lambda, eta, init_mode, w, b, True, max_epochs, beta1, beta2, eps)


    train_accuracy, yp = accuracy(X_train, y_train, w_n, b_n, n_layers, activation_f)
    validation_accuracy, yp = accuracy(X_validation, y_validation, w_n, b_n, n_layers, activation_f)
    test_accuracy, ytp = accuracy(X_test, y_test, w_n, b_n, n_layers, activation_f)

    cm = wandb.plot.confusion_matrix(y_true=y_test, preds=ytp, class_names=u_l)
    wandb.log({"conf_mat": cm})



    wandb.log({"train_acc": train_accuracy, \
               "validation_acc": validation_accuracy, \
                "test_acc": test_accuracy, \
                })
    # data = [[x, y] for (x, y) in zip(xxnn, trac)]
    # table = wandb.Table(data=data, columns = ["x", "y"])
    # wandb.log({"my_custom_plot_id" : wandb.plot.line(table,
    #                              "x", "y", title="train_accuracy_steps")})
    # data2 = [[x, y] for (x, y) in zip(xxnn, vaac)]
    # table2 = wandb.Table(data=data2, columns = ["x", "y"])
    # wandb.log({"my_custom_plot_id1" : wandb.plot.line(table2,
    #                              "x", "y", title="validation_accuracy_steps")})
    # data3 = [[x, y] for (x, y) in zip(xxnn, teac)]
    # table3 = wandb.Table(data=data3, columns = ["x", "y"])
    # wandb.log({"my_custom_plot_id2" : wandb.plot.line(table3,
    #                              "x", "y", title="test_accuracy_steps")})

    print("accu: ", validation_accuracy)




####################################################################

sweep_id = wandb.sweep(sweep_config, project = "Assignment1")
wandb.agent(sweep_id, function = train_nn, count=3)
wandb.finish()

#train_nn()
#wandb.finish()
